# Índice
1. [Introdução](#introduction)
2. [Pré-processamento dos dados](#pre-processing)

## 1. Introdução <a name="introduction"></a>
<p>Este <i>notebook</i> contém a utilização do algoritmo de regressão conhecido como [SGDRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html) - Gradiente Descendente Estocástico ou do inglês <i>Stochastic Gradient Descent.</i></p>

In [168]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns; sns.set() 
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer

## 2. Pré-processamento dos dados <a name="pre-processing"></a>
<p>Todo o passo-a-passo do pré-processamento dos dados pode ser encontrado no arquivo <i>rdu-weather-history.ipynb</i>, sendo que aqui o processo será bastante semelhante, porém de maneira reduzida e com modificações para adequar melhor à situação.</p>

In [160]:
weather_history = pd.read_csv('rdu-weather-history.csv', parse_dates=['date'], sep=';')
weather_history.shape

(4137, 28)

Substituindo as string por números binários:
* No = 0.0
* Yes = 1.0

In [161]:
weather_history = weather_history.replace('No', 0.0)
weather_history = weather_history.replace('Yes', 1.0)

<p>Esse é um caso em que o pré-processamento diverge do arquivo de pré-processamento. Isso aconteceu porque a abordagem utilizando esse modelo é diferente, não dividindo o dataset por estação do ano, e sim utilizando ele todo, mas substituindo a <i>feature season</i> por <i>dummies</i>. Essa escolha foi feita devido ao <i>SGDRegressor </i> ser mais indicado para datasets grandes, passando de 10000 linhas, por isso é mais interessante aplicá-lo em um de 4000 linhas do que em 4 de aproximadamente 1000 linhas.</p>

Substituindo os valores nulos pela média dos valores referentes a feature

In [162]:
imputer = Imputer(missing_values=np.nan, strategy='median', axis=0)
weather_history[['temperaturemin']] = imputer.fit_transform(weather_history[['temperaturemin']])
weather_history[['temperaturemax']] = imputer.fit_transform(weather_history[['temperaturemax']])
weather_history[['precipitation']] = imputer.fit_transform(weather_history[['precipitation']])
weather_history[['snowfall']] = imputer.fit_transform(weather_history[['snowfall']])
weather_history[['snowdepth']] = imputer.fit_transform(weather_history[['snowdepth']])
weather_history[['avgwindspeed']] = imputer.fit_transform(weather_history[['avgwindspeed']])
weather_history[['fastest2minwinddir']] = imputer.fit_transform(weather_history[['fastest2minwinddir']])
weather_history[['fastest2minwindspeed']] = imputer.fit_transform(weather_history[['fastest2minwindspeed']])
weather_history[['fastest5secwinddir']] = imputer.fit_transform(weather_history[['fastest5secwinddir']])
weather_history[['fastest5secwindspeed']] = imputer.fit_transform(weather_history[['fastest5secwindspeed']])

Criando coluna season a partir da data

In [163]:
season_array = []
for date in weather_history['date']:
    if int(date.strftime('%m%d')) >= 1221 or int(date.strftime('%m%d')) <= 319:
        season = 'winter'
    elif int(date.strftime('%m%d')) >= 320 and int(date.strftime('%m%d')) <= 620:
        season = 'spring'
    elif int(date.strftime('%m%d')) >= 621 and int(date.strftime('%m%d')) <= 921:
        season = 'summer'
    elif int(date.strftime('%m%d')) >= 922 and int(date.strftime('%m%d')) <= 1220:
        season = 'autumn'    
    
    season_array.append(season)

weather_history = weather_history.drop(columns=['date'])
weather_history['season'] = season_array

Substituindo a coluna season por dummies

In [164]:
weather_history = weather_history
dummies = pd.get_dummies(weather_history['season'])
weather_history = weather_history.drop(columns=['season'])
dummie_weather_history = pd.concat([weather_history, dummies], axis=1)

dummie_weather_history.shape

(4137, 31)